<a href="https://colab.research.google.com/github/Danil1D/KSVD/blob/main/%D0%94%D0%B0%D1%88%D0%BA%D0%B5%D0%B2%D0%B8%D1%87_%D0%9B%D0%B0%D0%B1_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
gpiosenka_cards_image_datasetclassification_path = kagglehub.dataset_download('gpiosenka/cards-image-datasetclassification')

print('Data source import complete.')



Data source import complete.


In [ ]:


# 📌 Імпорти
import os
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# 📁 Шляхи до папок
train_dir = '/kaggle/input/cards-image-datasetclassification/train'
val_dir = '/kaggle/input/cards-image-datasetclassification/valid'
test_dir = '/kaggle/input/cards-image-datasetclassification/test'

# 🔁 Трансформації
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# 📦 Датасети
train_data = datasets.ImageFolder(train_dir, transform=transform)
val_data = datasets.ImageFolder(val_dir, transform=transform)
test_data = datasets.ImageFolder(test_dir, transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)
test_loader = DataLoader(test_data, batch_size=32)

class_names = train_data.classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 🧠 Власна CNN-модель
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * 56 * 56, 256), nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.net(x)

model_cnn = SimpleCNN(num_classes=len(class_names)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_cnn.parameters(), lr=0.001)

# 🔁 Функція тренування
def train_model(model, train_loader, val_loader, epochs=5):
    train_loss, val_loss = [], []
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        train_loss.append(running_loss / len(train_loader))

        model.eval()
        val_running_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item()
        val_loss.append(val_running_loss / len(val_loader))

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss[-1]:.4f}, Val Loss: {val_loss[-1]:.4f}")
    return train_loss, val_loss

# 🔧 Навчання CNN
train_loss, val_loss = train_model(model_cnn, train_loader, val_loader, epochs=5)

# 📈 Криві навчання
plt.plot(train_loss, label='Train')
plt.plot(val_loss, label='Validation')
plt.title('Learning Curves (CNN)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# 🧪 Оцінка
def evaluate_model(model, loader):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.numpy())
            y_pred.extend(preds.cpu().numpy())
    return y_true, y_pred

# 📊 Матриця плутанини (CNN)
y_true, y_pred = evaluate_model(model_cnn, test_loader)
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(xticks_rotation=90)
plt.title("Confusion Matrix (CNN)")
plt.show()

# 🧠 Претренована VGG16
vgg16 = models.vgg16(pretrained=True)
for param in vgg16.parameters():
    param.requires_grad = False

vgg16.classifier[6] = nn.Linear(4096, len(class_names))
vgg16 = vgg16.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg16.classifier[6].parameters(), lr=0.001)

# 🔧 Навчання VGG16
train_model(vgg16, train_loader, val_loader, epochs=3)

# 📊 Матриця плутанини (VGG16)
y_true_vgg, y_pred_vgg = evaluate_model(vgg16, test_loader)
cm_vgg = confusion_matrix(y_true_vgg, y_pred_vgg)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_vgg, display_labels=class_names)
disp.plot(xticks_rotation=90)
plt.title("Confusion Matrix (VGG16)")
plt.show()

# 🎨 Візуалізація результатів
def visualize_predictions(model, loader, n=6):
    model.eval()
    shown = 0
    plt.figure(figsize=(15, 8))
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            for i in range(inputs.size(0)):
                if shown >= n:
                    break
                plt.subplot(2, 3, shown + 1)
                img = inputs[i].cpu().numpy().transpose((1, 2, 0))
                plt.imshow(img)
                plt.title(f"True: {class_names[labels[i]]}\nPred: {class_names[preds[i]]}")
                plt.axis('off')
                shown += 1
            if shown >= n:
                break
    plt.tight_layout()
    plt.show()

visualize_predictions(vgg16, test_loader)


KeyboardInterrupt: 